Data Engineering Project - Webscraping (ETL)
=============================================
In this project, I performed webscraping (Extraction), transformed data (Transformation), and saved it into my local drive (Load).

* Tool / Libraries: **Python** / pandas, numpy, requests, sqlite3, bs4, datetime

* Project Scenario: In the digital age, the film industry has witnessed exponential growth in both production and consumption. With an overwhelming number of movies being released each year, it becomes crucial for film enthusiasts, critics, and data analysts to have access to comprehensive and up-to-date film information. Extract, Transform, Load (ETL) processes play a vital role in collecting, organizing, and analyzing data from various sources. In this context, web scraping has emerged as a powerful technique to extract film data from online sources and enable the rating of films based on gathered information.


In [1]:
# Import Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import sqlite3
from datetime import datetime

### STEP01: Write a extraction function

The selected Python code is a function named extract that takes two parameters: url and table_attribs. This function is designed to scrape data from a webpage and store it in a pandas DataFrame for further processing.

In [2]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('table')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            name = col[1].find('a')
            name_text = name.text.strip() if name else ''
            data_dict = {"Name": name_text,
                         "Gross": col[7].text[1:],
                         "Release Date": col[8].text}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)
    return df

### STEP02: Write a transformation function

The selected Python code is a function named transform that takes a pandas DataFrame df as an argument. This function is designed to modify the 'Gross' column of the DataFrame by removing commas, converting the values to floats, and then scaling the values to millions.

In [3]:
def transform(df):

    film_list = df['Gross'].tolist()

    # Remove commas 
    film_list = [x.replace(',','') for x in film_list]

    # Now convert to floats
    film_list = [float(x) for x in film_list]

    film_list = [np.round(x/1000000,2) for x in film_list]

    df['Gross'] = film_list

    return df

### STEP03: Write a loading function (csv)
Load the transformed dataframe to an output CSV file. Write a function **load_to_csv()**, execute a function call and verify the output.

In [4]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path)

### STEP04: Write a loading function (database)
Load the transformed dataframe to an SQL database server as a table. Write a function **load_to_db()**, execute a function call and verify the output.

In [5]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


### STEP05: Write a a running query function
Run queries on the database table. Write a function **run_query()**, execute a given set of queries and verify the output.

In [6]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

### STEP06: Write a logging function
Write a function **log_progress()** to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

In [7]:
def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    with open("log.txt", "a") as log_file:
        log_file.write(timestamp + " - " + message + "\n")

### STEP07: Run code and check result

In [8]:
url = "https://www.boxofficemojo.com/year/2023/"
table_attribs = ["Name", "Gross", "Release Date"]
table_name = "Movies_Rating"
db_name = "Movies.db"
csv_path = "Movies.csv"

In [9]:
log_progress("ETL process started")
df = extract(url, table_attribs)

log_progress("Data extracted from the website")
df = transform(df)

log_progress("Data transform completed. Loading to CSV")
load_to_csv(df, csv_path)

log_progress("Loading to CSV completed. Loading to DB")
sql_connection = sqlite3.connect("film_Rating.db")

log_progress("Loading to DB completed. Running query")
load_to_db(df, sql_connection, table_name)

query_statement = f"SELECT * FROM {table_name}"
run_query(query_statement, sql_connection)

log_progress("ETL process completed")
sql_connection.close()


SELECT * FROM Movies_Rating
                                    Name   Gross Release Date
0                                 Barbie  636.23       Jul 21
1            The Super Mario Bros. Movie  574.93        Apr 5
2    Spider-Man: Across the Spider-Verse  381.31        Jun 2
3         Guardians of the Galaxy Vol. 3  359.00        May 5
4                            Oppenheimer  326.10       Jul 21
..                                   ...     ...          ...
195                Knights of the Zodiac    1.09       May 12
196                               Broker    1.05       Dec 23
197                           Showing Up    0.75        Apr 7
198                               Inside    0.92       Mar 17
199              The Roundup: No Way Out    0.91        Jun 2

[200 rows x 3 columns]
